# Virtual memory

Process should provide each program with the following abstractions:
* Independent control flow
    * Each process should think that it has exclusive use of the CPU
* Private virtual address space
    * Each process should think that it has exclusive use of main memory

Note that the processes has to think that it is that way, but it could be false in actuality.
This is achieved using a **virtual memory system**.

## Goals of virtual memory

### Fit large memory into small physical memory

Consider a 32-bit system.
There is $2^{32} \approx 4GB$ worth of addresses.
Now, consider the physical memory.
For modern physical memory, this is rather possible to fit that amount of addresses into the main memory.

Now consider a 64-bit system instead.
The amount of address is $2^{64} \approx 16EB$ (exabytes).
This is impossible to fit inside the physical main memory.

### Manage memory space of multiple processes

Virtual memory decides where the process memory (stack, heap, data) gets allocated on the main memory.

### Protect processes from accessing another's memory

By ensuring that a process's memory is not mapped to the same location of another process in the physical memory, we can enforce that no illegal memory access happens.

### Allow processes to access another's memory

Similarly, if multiple processes need to share some memory, then we map these virtual addresses to the same location in the physical memory.

By performing the above, the processes has no idea that the physical memory is being shared across multiple processes, hence forming a layer of abstraction.

## Implementation

Virtual memory is simply a mapping from virtual addresses (of the process) to physical addresses (of the main memory).

The virtual memory (and physical memory) is partitioned into smaller blocks called **pages**, where a page of size $p$ occupies $2^p$ bytes.

Some pages on the virtual memory is cached in the physical main memory (DRAM) for faster access.

### Status of page

A virtual memory page can be in either of these states:
* Unallocated: Page is not being used by process
* Uncached: Page is being used by process, but not cached in DRAM
    * This means it is mapped to the disk
* Cached: Page is being used by processes and is cached in DRAM

Note that disk access is much slower than DRAM, thus we would like to avoid it as much as possible, hence the caching.

### Page table

Each process has its own **page table**, which is stored in the DRAM.
Since we are working with pages, we need a way of mapping virtual pages to physical pages.
Each entry in the page table contains:
* a bit which indicates whether the page is cached in DRAM
* the physical page number, if it is cached in DRAM

For example:

| Virtual page number | Valid bit | Physical page number |
| --- | --- | --- |
| 0 | 1 | 0x10 |
| 1 | 0 | disk |
| 2 | 1 | 0x13 |
| ... | ... | ... | 
| 256 | 1 | 0x42 |

Hence, consider the following 14-bit virtual memory address:


```
0b00000010010110 
```

Suppose that our page size is 64 bytes.
That means there are $2^{14}B/64B = 256$ virtual pages, and the last $\log_2{64} = 6$ bits in the virtual address is used to determine the **page offset**, which is the address within a page.

This also means that our virtual page number requires $14 - 6 = 8$ bits.

Thus, we can view our address as :

| Virtual page number | page offset |
| :---: | :---: |
| `00000010`            |      `010110` |

Suppose that our physical memory is 12-bit.

Then we need to find our physical page number (which has $12 - 6 = 6$ bits).
Since the virtual page number is `0b00000010 = 2`, the corresponding physical page number is `0x13`.
Converting this to 6-bit, we get the following address:

| Physical page number | page offset |
| :---: | :---: |
| `001101`            |      `010110` |

Note that the page offset is unchanged.

### Page hit

The previous example is an example of a page hit.
This means that we referenced a virtual memory address that is in physical memory.

In more detail, the following occurs:
1. CPU requests a virtual memory by its address from the memory management unit (MMU)
2. MMU fetches the page table entry in the DRAM
3. Using the page table entry, MMU computes the physical address and sends it to the MMU
4. MMU respond to the CPU with the data at that address

### Page fault

A page fault occurs when the virtual memory address is not in the physical memory.

In more detail, the following occurs:
1. CPU requests a virtual memory by its address from the MMU
2. MMU fetches the page table entry in the DRAM
3. MMU notices that the valid bit is 0
4. MMU triggers a page fault exception
5. Page fault handler decides on a **victim** page to evict from DRAM
6. Handler fetches desired page from disk and update page table entry
7. Handler returns control to original processes
8. Repeats from faulty instruction, where the [page hit flow](#Page-hit) will occur.

### Translation look-aside buffer

Notice that the MMU has to keep accessing DRAM to obtain the page table entry.
To avoid this overhead, MMU usually comes with a small hardware cache, called the **translation look-aside buffer** which contains a subset of the page table entry.

## Benefits


With virtual memory, we are able to sidestep the requirement of [contiguous memory](./memory_allocation.ipynb#contiguous-memory), as processes now thinks they are working with contiguous memory, but the physical memory could be segmented.

It also adds a further layer of abstraction above the process level to allow easier implementation of memory management.